In [2]:
from utils import *
from dataset  import *
from models import *

In [15]:
test_file = "test_data_iob.csv"
val_file = "val_data_iob.csv"


In [4]:
pretrained_model = "recobo/chemical-bert-uncased-pharmaceutical-chemical-classifier"
batch_size = 4
max_para_length = 128
para_seq_len = 16  #number of paras to be encoded and decoded together (hyperparameter)
# Check if cuda is available and set device
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")

# Make sure you choose suitable num_worker, otherwise it will result in errors
num_workers = 8 if cuda else 0

print("Cuda = ", str(cuda), " with num_workers = ", str(num_workers),  " system version = ", sys.version)

Cuda =  True  with num_workers =  8  system version =  3.7.13 (default, Oct 18 2022, 18:57:03) 
[GCC 11.2.0]


In [16]:
test_data = CRFEmbeddingDataset(test_file, para_seq_len = para_seq_len, pretrained_model = pretrained_model, stride = para_seq_len)
test_args = dict(shuffle=False, batch_size=batch_size, num_workers=8, pin_memory=True, drop_last=False) if cuda else dict(shuffle=False, batch_size=batch_size, drop_last=False)
test_loader = DataLoader(test_data, **test_args)

val_data = CRFEmbeddingDataset(val_file, para_seq_len = para_seq_len, pretrained_model = pretrained_model, stride = para_seq_len)
val_args = dict(shuffle=False, batch_size=batch_size, num_workers=8, pin_memory=True, drop_last=False) if cuda else dict(shuffle=False, batch_size=batch_size, drop_last=False)
val_loader = DataLoader(val_data, **val_args)



128
128


In [17]:

print(val_data.__len__())
print(test_data.__len__())

print(len(test_loader))
print(len(val_loader))

471
804
201
118


## Model 1
./chem_bert_iob_bilstm_crf_bert_finetune/enc_dec_model_model_params_0.9525667530577686.pth

pretrained_model = "recobo/chemical-bert-uncased-pharmaceutical-chemical-classifier"

batch_size = 4

max_para_length = 128

para_seq_len = 16  #number of paras to be encoded and decoded together (hyperparameter)

'./model_model_params_0.9428545098368426.pth'


In [7]:
model = EncoderDecoderBiLstmCRF(embed_model = BertEmbedding(pretrained_model), num_tags = 3, freeze_bert=False)
model.load_state_dict(torch.load('./chem_bert_iob_bilstm_crf_bert_finetune/enc_dec_model_model_params_0.9525667530577686.pth'))
# model = load_pretrained_weights(model, './model_model_params_0.9428545098368426.pth')

# if torch.cuda.device_count() > 1:
#   print("Let's use", torch.cuda.device_count(), "GPUs!")
#   model = nn.DataParallel(model)
model = model.to(device)

total_params = sum(p.numel() for p in model.parameters())
trainable_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
non_trainable_total_params = sum(p.numel() for p in model.parameters() if not p.requires_grad)
print("Total params: ", total_params)
print("Trainable params: ", trainable_total_params)
print("Non Trainable params: ", non_trainable_total_params)


epochs = 10 #changed from 10
lamda = 1e-3  #L2 regularization (prev : 1e-4)
learning_rate = 5e-5 #changed from 1e-2   ## Greatly reduces LR for bert finetuning

# criterion = nn.CrossEntropyLoss()
# criterion = criterion.to(device)

optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=lamda)
# optimizer.load_state_dict(torch.load('./bert_base_triplet/optimizer_model_params_0.9409211846833226.pth'))    

# scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[i for i in range(4,20,4)], gamma=0.75)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=(len(test_loader) * epochs))


Some weights of the model checkpoint at recobo/chemical-bert-uncased-pharmaceutical-chemical-classifier were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Total params:  130909458
Trainable params:  130909458
Non Trainable params:  0


In [32]:
val_df = pd.read_csv(val_file)
_, val_predictions = validate(model, val_loader, device, optimizer, scheduler)
_ = get_span_perf(val_df, val_predictions)


learning_rate: [5e-05]
Validation loss: 4.65, Time: 17.07770848274231
{'eval_accuracy': 0.9532908704883227, 'eval_precision': 0.952948854512343, 'eval_recall': 0.9532908704883227, 'eval_f-1': 0.9521363436342459}
Total original spans:  911
Total predicted spans:  758
Total number of original spans correctly predicted acc to strict match:  672
Percent of original spans correctly predicted acc to strict match:  73.76509330406147
Total number of original spans correctly predicted acc to fuzzy match:  729
Percent of original spans correctly predicted acc to fuzzy match:  80.02195389681668
Count of fuzzy matched spans:  57
Count of spans with misaligned begin and end: 21 (36.84%) 
Count of spans with misaligned begin: 15 (26.32%) 
Count of spans with misaligned end: 21 (36.84%) 


In [33]:
test_df = pd.read_csv(test_file)
_, test_predictions = validate(model, test_loader, device, optimizer, scheduler)
_ = get_span_perf(test_df, test_predictions)


learning_rate: [5e-05]
Validation loss: 5.04, Time: 29.1264865398407
{'eval_accuracy': 0.9406094527363185, 'eval_precision': 0.9394594020800763, 'eval_recall': 0.9406094527363185, 'eval_f-1': 0.9397341370157223}
Total original spans:  1234
Total predicted spans:  1104
Total number of original spans correctly predicted acc to strict match:  854
Percent of original spans correctly predicted acc to strict match:  69.20583468395462
Total number of original spans correctly predicted acc to fuzzy match:  962
Percent of original spans correctly predicted acc to fuzzy match:  77.9578606158833
Count of fuzzy matched spans:  108
Count of spans with misaligned begin and end: 13 (12.04%) 
Count of spans with misaligned begin: 26 (24.07%) 
Count of spans with misaligned end: 69 (63.89%) 


## Model 2

./chem_bert_iob_bilstm_crf_bert_finetune/enc_dec_model_model_params_0.953476708995007.pth


In [26]:
model = EncoderDecoderBiLstmCRF(embed_model = BertEmbedding(pretrained_model), num_tags = 3, freeze_bert=False)
model.load_state_dict(torch.load('./chem_bert_iob_bilstm_crf_bert_finetune/enc_dec_model_model_params_0.953476708995007.pth'))
# model = load_pretrained_weights(model, './model_model_params_0.9428545098368426.pth')

# if torch.cuda.device_count() > 1:
#   print("Let's use", torch.cuda.device_count(), "GPUs!")
#   model = nn.DataParallel(model)
model = model.to(device)

total_params = sum(p.numel() for p in model.parameters())
trainable_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
non_trainable_total_params = sum(p.numel() for p in model.parameters() if not p.requires_grad)
print("Total params: ", total_params)
print("Trainable params: ", trainable_total_params)
print("Non Trainable params: ", non_trainable_total_params)


epochs = 10 #changed from 10
lamda = 1e-3  #L2 regularization (prev : 1e-4)
learning_rate = 5e-5 #changed from 1e-2   ## Greatly reduces LR for bert finetuning

# criterion = nn.CrossEntropyLoss()
# criterion = criterion.to(device)

optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=lamda)
# optimizer.load_state_dict(torch.load('./bert_base_triplet/optimizer_model_params_0.9409211846833226.pth'))    

# scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[i for i in range(4,20,4)], gamma=0.75)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=(len(test_loader) * epochs))


Some weights of the model checkpoint at recobo/chemical-bert-uncased-pharmaceutical-chemical-classifier were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Total params:  130909458
Trainable params:  130909458
Non Trainable params:  0


In [27]:
val_df = pd.read_csv(val_file)
_, val_predictions = validate(model, val_loader, device, optimizer, scheduler)
_ = get_span_perf(val_df, val_predictions)


learning_rate: [5e-05]
Validation loss: 2.80, Time: 16.683980464935303
{'eval_accuracy': 0.9538216560509554, 'eval_precision': 0.9534230759626194, 'eval_recall': 0.9538216560509554, 'eval_f-1': 0.953476708995007}
Total original spans:  911
Total predicted spans:  861
Total number of original spans correctly predicted acc to strict match:  706
Percent of original spans correctly predicted acc to strict match:  77.49725576289791
Total number of original spans correctly predicted acc to fuzzy match:  764
Percent of original spans correctly predicted acc to fuzzy match:  83.86388583973655
Count of fuzzy matched spans:  58
Count of spans with misaligned begin and end: 28 (48.28%) 
Count of spans with misaligned begin: 14 (24.14%) 
Count of spans with misaligned end: 16 (27.59%) 


In [28]:
test_df = pd.read_csv(test_file)
_, test_predictions = validate(model, test_loader, device, optimizer, scheduler)
_ = get_span_perf(test_df, test_predictions)


learning_rate: [5e-05]
Validation loss: 4.07, Time: 28.625789642333984
{'eval_accuracy': 0.9465174129353234, 'eval_precision': 0.945344867870555, 'eval_recall': 0.9465174129353234, 'eval_f-1': 0.9455013765218424}
Total original spans:  1234
Total predicted spans:  1102
Total number of original spans correctly predicted acc to strict match:  874
Percent of original spans correctly predicted acc to strict match:  70.82658022690438
Total number of original spans correctly predicted acc to fuzzy match:  993
Percent of original spans correctly predicted acc to fuzzy match:  80.47001620745543
Count of fuzzy matched spans:  119
Count of spans with misaligned begin and end: 23 (19.33%) 
Count of spans with misaligned begin: 31 (26.05%) 
Count of spans with misaligned end: 65 (54.62%) 


## Model 3

/home/anjadhav/Chemical-Patent-Reaction-Extraction/chem_bert_iob_bilstm_crf_bert_finetune/model_model_params_0.9521363436342459.pth

In [29]:
model = EncoderDecoderBiLstmCRF(embed_model = BertEmbedding(pretrained_model), num_tags = 3, freeze_bert=False)
model.load_state_dict(torch.load('./chem_bert_iob_bilstm_crf_bert_finetune/model_model_params_0.9521363436342459.pth'))
# model = load_pretrained_weights(model, './model_model_params_0.9428545098368426.pth')

# if torch.cuda.device_count() > 1:
#   print("Let's use", torch.cuda.device_count(), "GPUs!")
#   model = nn.DataParallel(model)
model = model.to(device)

total_params = sum(p.numel() for p in model.parameters())
trainable_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
non_trainable_total_params = sum(p.numel() for p in model.parameters() if not p.requires_grad)
print("Total params: ", total_params)
print("Trainable params: ", trainable_total_params)
print("Non Trainable params: ", non_trainable_total_params)


epochs = 10 #changed from 10
lamda = 1e-3  #L2 regularization (prev : 1e-4)
learning_rate = 5e-5 #changed from 1e-2   ## Greatly reduces LR for bert finetuning

# criterion = nn.CrossEntropyLoss()
# criterion = criterion.to(device)

optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=lamda)
# optimizer.load_state_dict(torch.load('./bert_base_triplet/optimizer_model_params_0.9409211846833226.pth'))    

# scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[i for i in range(4,20,4)], gamma=0.75)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=(len(test_loader) * epochs))


Some weights of the model checkpoint at recobo/chemical-bert-uncased-pharmaceutical-chemical-classifier were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Total params:  130909458
Trainable params:  130909458
Non Trainable params:  0


In [30]:
val_df = pd.read_csv(val_file)
_, val_predictions = validate(model, val_loader, device, optimizer, scheduler)
_ = get_span_perf(val_df, val_predictions)


learning_rate: [5e-05]
Validation loss: 4.65, Time: 17.00141930580139
{'eval_accuracy': 0.9532908704883227, 'eval_precision': 0.952948854512343, 'eval_recall': 0.9532908704883227, 'eval_f-1': 0.9521363436342459}
Total original spans:  911
Total predicted spans:  758
Total number of original spans correctly predicted acc to strict match:  672
Percent of original spans correctly predicted acc to strict match:  73.76509330406147
Total number of original spans correctly predicted acc to fuzzy match:  729
Percent of original spans correctly predicted acc to fuzzy match:  80.02195389681668
Count of fuzzy matched spans:  57
Count of spans with misaligned begin and end: 21 (36.84%) 
Count of spans with misaligned begin: 15 (26.32%) 
Count of spans with misaligned end: 21 (36.84%) 


In [31]:
test_df = pd.read_csv(test_file)
_, test_predictions = validate(model, test_loader, device, optimizer, scheduler)
_ = get_span_perf(test_df, test_predictions)


learning_rate: [5e-05]
Validation loss: 5.04, Time: 28.995968103408813
{'eval_accuracy': 0.9406094527363185, 'eval_precision': 0.9394594020800763, 'eval_recall': 0.9406094527363185, 'eval_f-1': 0.9397341370157223}
Total original spans:  1234
Total predicted spans:  1104
Total number of original spans correctly predicted acc to strict match:  854
Percent of original spans correctly predicted acc to strict match:  69.20583468395462
Total number of original spans correctly predicted acc to fuzzy match:  962
Percent of original spans correctly predicted acc to fuzzy match:  77.9578606158833
Count of fuzzy matched spans:  108
Count of spans with misaligned begin and end: 13 (12.04%) 
Count of spans with misaligned begin: 26 (24.07%) 
Count of spans with misaligned end: 69 (63.89%) 


## Model 4
/home/anjadhav/Chemical-Patent-Reaction-Extraction/chem_bert_iob_bilstm_crf/model_model_params_0.9539811224849719.pth

In [ ]:
model = EncoderDecoderBiLstmCRF(embed_model = BertEmbedding(pretrained_model), num_tags = 3, freeze_bert=False)
model.load_state_dict(torch.load('./chem_bert_iob_bilstm_crf/model_model_params_0.9539811224849719.pth'))
# model = load_pretrained_weights(model, './model_model_params_0.9428545098368426.pth')

# if torch.cuda.device_count() > 1:
#   print("Let's use", torch.cuda.device_count(), "GPUs!")
#   model = nn.DataParallel(model)
model = model.to(device)

total_params = sum(p.numel() for p in model.parameters())
trainable_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
non_trainable_total_params = sum(p.numel() for p in model.parameters() if not p.requires_grad)
print("Total params: ", total_params)
print("Trainable params: ", trainable_total_params)
print("Non Trainable params: ", non_trainable_total_params)


epochs = 10 #changed from 10
lamda = 1e-3  #L2 regularization (prev : 1e-4)
learning_rate = 5e-5 #changed from 1e-2   ## Greatly reduces LR for bert finetuning

# criterion = nn.CrossEntropyLoss()
# criterion = criterion.to(device)

optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=lamda)
# optimizer.load_state_dict(torch.load('./bert_base_triplet/optimizer_model_params_0.9409211846833226.pth'))    

# scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[i for i in range(4,20,4)], gamma=0.75)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=(len(test_loader) * epochs))


In [35]:
val_df = pd.read_csv(val_file)
_, val_predictions = validate(model, val_loader, device, optimizer, scheduler)
_ = get_span_perf(val_df, val_predictions)


Epoch:   6%|▌         | 7/118 [00:01<00:17,  6.44it/s]